The [Applications API](https://dev.writer.com/api-guides/applications) allows you to turn deployed [no-code applications](https://dev.writer.com/no-code/introduction) into microservices and tools.

This cookbook shows how to use the Applications API to invoke deployed applications.

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- A [no-code application deployed in Writer AI Studio](https://dev.writer.com/no-code/introduction)
    - If you don't have an application deployed, follow [this guide](https://dev.writer.com/no-code/building-a-text-generation-app) to build and deploy a text generation application.
    - Store the application ID for later use in this tutorial. You can find the application ID in the URL of the application in the Writer AI Studio UI. The URL structure is: `https://app.writer.com/aistudio/organization/<organization-id>/app/<app-id>`.

## Setup

Install the Writer SDK:


In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

You've now initialized the Writer client and can use it to invoke deployed applications.

### Inputs definition

Each no-code application has a set of inputs that you need to provide when invoking the application via the API. The inputs are defined in the application's settings, which you can see via the API or in the Writer AI Studio UI.

This guide shows how to get the inputs for an application via the API.

Enter the ID for the application you want to use in the following cell:

In [ ]:
application_id = input("Enter ID of Application you want to use: ")

Next, retrieve the required inputs for the application:

In [ ]:
application = client.applications.retrieve(application_id)
for input_ in application.inputs:
    print(f"{input_.name} ({input_.input_type}): {input_.description}")

Now that you know the inputs to the application, you can create an array of inputs with the values you want to use when invoking the application. The inputs array should be in the following format:

```python
inputs=[
    {
        "id": "ID (name) of input field",
        "value": [
            "Value of input field"
        ]
    }
]
```

The following cell loops through the inputs in your application and creates an array of inputs with the values you want to use when invoking the application.

In [ ]:
inputs = []
for input_ in application.inputs:
    inputs.append({
        "id": input_.name,
        "value": [input(f"Enter value for {input_.name} (type: {input_.input_type}): ")]
    })
print(f"Inputs: {inputs}")

Now you can invoke the application with the inputs you've defined:

In [ ]:
application_generate_content_response = client.applications.generate_content(
    application_id=application_id,
    inputs=inputs
)

print(application_generate_content_response.suggestion)

### Streaming

[Research Assistant](https://dev.writer.com/no-code/building-a-research-assistant-app) apps support streaming responses. To follow the next example, you'll need a deployed Research Assistant app in AI Studio. If you don't have one, you can follow [this guide](https://dev.writer.com/no-code/building-a-research-assistant-app) to build and deploy a research assistant application.

Research Assistant apps take a single input, which is the user's query. The input array is structured as follows:

```python
inputs=[
    {"id": "query", "value": [user_query]}
]
```

The following cell retrieves the Research Assistant app you want to use and asks you to fill in the query for the input array.

In [ ]:
research_app_id = input("Enter ID of Research Assistant app you want to use: ")
research_application = client.applications.retrieve(research_app_id)
research_inputs = [{
        "id": "query",
        "value": [input("Enter the research query: ")]
    }
]

Next, invoke the research assistant application with the query you've defined and stream the response by setting the `stream` parameter to `True`. The stream ends when the response is an empty string.

In [ ]:
application_generate_content_response = client.applications.generate_content(
    application_id=research_app_id,
    inputs=research_inputs,
    stream=True  # streaming is only available for research assistant apps
)

for chunk in application_generate_content_response:
    delta = chunk.delta
    if delta.title:
        print(delta.title)
    if delta.stages:
        print(delta.stages[0].content)
    if delta.content:
        print(delta.content, end="", flush=True)

### Retrieve a list of apps

Use the `list` method to retrieve a list of all your deployed applications:

In [ ]:
page = client.applications.list()

for app in page.data:
    print(f"{app.name} {app.id}")

The `list` method supports pagination. You can retrieve a subset of your applications by specifying the [`order`](https://dev.writer.com/api-guides/api-reference/application-api/list-applications#parameter-order), [`after`](https://dev.writer.com/api-guides/api-reference/application-api/list-applications#parameter-after), [`before`](https://dev.writer.com/api-guides/api-reference/application-api/list-applications#parameter-before), and [`limit`](https://dev.writer.com/api-guides/api-reference/application-api/list-applications#parameter-limit) parameters. You can also filter the applications by [`type`](https://dev.writer.com/api-guides/api-reference/application-api/list-applications#parameter-type).

The following cell retrieves the first two applications in ascending order, after the application with the ID specified by `application_id`.

In [ ]:
page_with_params = client.applications.list(order="asc", after=application_id, limit=2)

for app in page_with_params.data:
    print(f"{app.name} {app.id}")